In [7]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [8]:
import os
plt.style.use(os.path.join(os.getcwd(), 'mystyle.mplstyle') )
plt.rcParams['axes.edgecolor'] = 'w'

# Time Series analysis

In [9]:
import numpy as np

In [10]:
from pandas.io import data, wb
import scipy.stats as st
from statsmodels.tsa import stattools as stt
from statsmodels import tsa
import statsmodels.api as smapi
import datetime

ImportError: The pandas.io.data module is moved to a separate package (pandas-datareader). After installing the pandas-datareader package (https://github.com/pandas-dev/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.

## Introduction

In [ ]:
def despine(axs):
    # to be able to handle subplot grids
    # it assumes the input is a list of 
    # axes instances, if it is not a list, 
    # it puts it in one
    if type(axs) != type([]):
        axs = [axs]
    for ax in axs:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        ax.spines['bottom'].set_position(('outward', 10))
        ax.spines['left'].set_position(('outward', 10))

In [ ]:
def is_stationary(df, maxlag=14, autolag=None, regression='ct'):
    """Run the Augmented Dickey-Fuller test from statsmodels
    and print output.
    """
    outpt = stt.adfuller(df,maxlag=maxlag, autolag=autolag,
                            regression=regression)
    print('adf\t\t {0:.3f}'.format(outpt[0]))
    print('p\t\t {0:.3g}'.format(outpt[1]))
    print('crit. val.\t 1%: {0:.3f}, \
5%: {1:.3f}, 10%: {2:.3f}'.format(outpt[4]["1%"], 
                                     outpt[4]["5%"], outpt[4]["10%"]))
    print('stationary?\t {0}'.format(['true', 'false']\
                                   [outpt[0]>outpt[4]['5%']]))
    return outpt

## example 1 - trends, cycles and seasons

### the data

In [ ]:
%more data/mean-daily-temperature-fisher-river.csv

In [ ]:
dateparse = lambda d: pd.datetime.strptime(d, '%Y-%m-%d')

In [ ]:
temp = pd.read_csv('data/mean-daily-temperature-fisher-river.csv',
                   parse_dates=['Date'], 
                   index_col='Date', 
                   date_parser=dateparse,
                   )

In [ ]:
temp.info()

In [ ]:
temp.head()

In [ ]:
temp = temp.iloc[:,0]

In [ ]:
type(temp)

In [ ]:
plt.plot(temp)

In [ ]:
temp.head()

In [ ]:
temp.index

In [ ]:
temp.dtypes

In [ ]:
temp['1988-01']

In [ ]:
temp.plot(lw=1.5)
despine(plt.gca())
plt.gcf().autofmt_xdate()
plt.ylabel('Temperature');

In [ ]:
temp.describe()

### Slicing

In [ ]:
temp['1988'].plot(lw=1.5)
despine(plt.gca())
plt.gcf().autofmt_xdate()
plt.minorticks_off()

In [ ]:
temp['1988-01'].plot(ls='dotted', marker='.')
despine(plt.gca())
plt.gcf().autofmt_xdate()
plt.ylabel('Temperature');

In [ ]:
temp[temp < -25].head()

In [ ]:
temp['1988'].index

In [ ]:
kws = dict(lw=1.5, alpha=0.5)
plt.plot(temp['1988'].values, **kws)
plt.plot(temp['1989'].values, **kws)
plt.plot(temp['1990'].values, **kws)
plt.plot(temp['1991'].values, **kws)
despine(plt.gca())
plt.xlabel('Day of the year')
plt.ylabel('Temperature')
plt.title('Fisher River Mean Temperature');

### Resampling, smooting, rolling means

In [ ]:
temp.resample('A').head()

In [ ]:
pd.__version__

In [ ]:
temp.plot(lw=1.5, color='SkyBlue')
temp.resample('W').plot(lw=1, color='Green')
temp.resample('AS', loffset='178 D').plot(color='k')
plt.ylim(-50,30)
plt.ylabel('Temperature')
plt.title('Fisher River Mean Temperature')
plt.legend(['Raw', 'Binned Weekly', 'Binned Yearly'], loc=3)
despine(plt.gca());

In [ ]:
temp.plot(lw=1, alpha=0.7)
temp.resample('M').plot(lw=1.5, zorder=32)
temp.resample('6M').plot(lw=1.5, dashes=(5,2), 
                         zorder=32, color='Green')
plt.gcf().autofmt_xdate()
despine(plt.gca())
plt.ylim(-50,30)
plt.ylabel('Temperature')
plt.title('Fisher River Temperature')
plt.legend(['Raw', 'Binned 6 Months ', 'Binned 30 Days'], loc=3);

In [ ]:
temp.plot(lw=1, alpha=0.5)
pd.rolling_mean(temp, center=True, window=60).plot(color='Green')
plt.fill_between(temp.resample('M', label='left', 
                               loffset='15 D').index,
                 y1=temp.resample('M', how='max').values,
                 y2=temp.resample('M', how='min').values,
                 color='0.85',
                 )
plt.gcf().autofmt_xdate()
plt.ylabel('Temperature')
despine(plt.gca())
plt.title('Fisher River Temperature');

In [ ]:
pd.rolling_cov(temp, center=True, window=10).plot(color='Green',
                                                 lw=1.5)
despine(plt.gca());

In [ ]:
pd.rolling_var(temp, center=True, window=14).plot(color='Green',
                                                 lw=1.5)
despine(plt.gca());

In [ ]:
temp.index

---

this below is not going in book atm.

In [ ]:
temp_residual = temp-pd.rolling_mean(temp, center=True, window=60)

In [ ]:
temp_residual.plot(lw=1.5, color='Coral')
despine(plt.gca())
plt.gcf().autofmt_xdate()
plt.title('Residuals')
plt.ylabel('Temperature');

In [ ]:
pd.rolling_var(temp_residual, center=True, window=14).plot(color='Green',
                                                 lw=1.5)
despine(plt.gca());

## Stationarity, seasonality, trends etc

In [ ]:
def is_stationary(df, maxlag=14, autolag=None, regression='ct'):
    """Run the Augmented Dickey-Fuller test from statsmodels
    and print output.
    """
    outpt = stt.adfuller(df,maxlag=maxlag, autolag=autolag,
                            regression=regression)
    print('adf\t\t {0:.3f}'.format(outpt[0]))
    print('p\t\t {0:.3g}'.format(outpt[1]))
    print('crit. val.\t 1%: {0:.3f}, \
5%: {1:.3f}, 10%: {2:.3f}'.format(outpt[4]["1%"], 
                                     outpt[4]["5%"], outpt[4]["10%"]))
    print('stationary?\t {0}'.format(['true', 'false']\
                                   [outpt[0]>outpt[4]['5%']]))
    return outpt

### the data

In [ ]:
%more data/monthly-car-sales-in-quebec-1960.csv

In [ ]:
carsales = pd.read_csv('data/monthly-car-sales-in-quebec-1960.csv',
                   parse_dates=['Month'], 
                   index_col='Month', 
                   date_parser=lambda d: pd.datetime.strptime(d, '%Y-%m'),
                   )

In [ ]:
carsales.head()

In [ ]:
carsales = carsales.iloc[:,0]

In [ ]:
carsales.index

In [ ]:
plt.plot(carsales)
despine(plt.gca())
plt.gcf().autofmt_xdate()
plt.xlabel('Year')
plt.ylabel('Sales')
plt.xlim('1960','1969')
plt.title('Monthly Car Sales');

### testing for stationarity

In [ ]:
is_stationary(carsales);

### decomposing time series components

#### with statsmodels.seasonal decompose

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
carsales_decomp = seasonal_decompose(carsales, freq=12)

In [ ]:
carsales_trend = carsales_decomp.trend
carsales_seasonal = carsales_decomp.seasonal
carsales_residual = carsales_decomp.resid

In [ ]:
def change_plot(ax):
    despine(ax)
    ax.locator_params(axis='y', nbins=5)
    plt.setp(ax.get_xticklabels(), rotation=90, ha='center')

plt.figure(figsize=(9,4.5))

plt.subplot(221)
plt.plot(carsales, color='Green')
change_plot(plt.gca())
plt.title('Sales', color='Green')
xl = plt.xlim()
yl = plt.ylim()

plt.subplot(222)
plt.plot(carsales.index,carsales_trend, 
         color='Coral')
change_plot(plt.gca())
plt.title('Trend', color='Coral')
plt.gca().yaxis.tick_right()
plt.gca().yaxis.set_label_position("right")
plt.xlim(xl)
plt.ylim(yl)

plt.subplot(223)
plt.plot(carsales.index,carsales_seasonal, 
         color='SteelBlue')
change_plot(plt.gca())
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_major_formatter(plt.NullFormatter())
plt.xlabel('Seasonality', color='SteelBlue', labelpad=-20)
plt.xlim(xl)
plt.ylim((-8000,8000))

plt.subplot(224)
plt.plot(carsales.index,carsales_residual,
        color='IndianRed')
change_plot(plt.gca())
plt.xlim(xl)
plt.gca().yaxis.tick_right()
plt.gca().yaxis.set_label_position("right")
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_major_formatter(plt.NullFormatter())
plt.ylim((-8000,8000))
plt.xlabel('Residuals', color='IndianRed', labelpad=-20)

plt.tight_layout()
plt.subplots_adjust(hspace=0.55)

In [ ]:
fig = plt.figure(figsize=(7,1.5) )

ax1 = fig.add_axes([0.1,0.1,0.6,0.9])
ax1.plot(carsales-carsales_trend, 
         color='Green', label='Detrended data')
ax1.plot(carsales_seasonal, 
         color='Coral', label='Seasonal component')
kwrds=dict(lw=1.5, color='0.6', alpha=0.8)
d1 = pd.datetime(1960,9,1)
dd = pd.Timedelta('365 Days')
[ax1.axvline(d1+dd*i, dashes=(3,5),**kwrds) for i in range(9)]
d2 = pd.datetime(1960,5,1)
[ax1.axvline(d2+dd*i, dashes=(2,2),**kwrds) for i in range(9)]
ax1.set_ylim((-12000,10000))

ax1.locator_params(axis='y', nbins=4)
ax1.set_xlabel('Year')
ax1.set_title('Sales Seasonality')
ax1.set_ylabel('Sales')
ax1.legend(loc=0, ncol=2, frameon=True);

ax2 = fig.add_axes([0.8,0.1,0.4,0.9])
ax2.plot(carsales_seasonal['1960':'1960'], 
         color='Coral', label='Seasonal component')
ax2.set_ylim((-12000,10000))
[ax2.axvline(d1+dd*i, dashes=(3,5),**kwrds) for i in range(1)]
d2 = pd.datetime(1960,5,1)
[ax2.axvline(d2+dd*i, dashes=(2,2),**kwrds) for i in range(1)]
despine([ax1, ax2])

import matplotlib.dates as mpldates
yrsfmt = mpldates.DateFormatter('%b')
ax2.xaxis.set_major_formatter(yrsfmt)
labels = ax2.get_xticklabels()
plt.setp(labels, rotation=90);

In [ ]:
carsales_seasonal_component = carsales_seasonal['1960'].values

In [ ]:
carsales_residual.dropna(inplace=True)

In [ ]:
is_stationary(carsales_residual);

In [ ]:
loc, shape = st.norm.fit(carsales_residual)
x=range(-3000,3000)
y = st.norm.pdf(x, loc, shape)
n, bins, patches = plt.hist(carsales_residual, bins=20, normed=True)
plt.plot(x,y, color='Coral')
despine(plt.gca())
plt.title('Residuals')
plt.xlabel('Value'); plt.ylabel('Counts');

In [ ]:
(osm,osr), (slope, intercept, r) = st.probplot(carsales_residual, dist='norm', fit=True)
line_func = lambda x: slope*x + intercept
plt.plot(osm,osr,
         '.', label='Data', color='Coral')
plt.plot(osm, line_func(osm), 
         color='SteelBlue',
         dashes=(20,5), label='Fit')
plt.xlabel('Quantiles'); plt.ylabel('Ordered Values')
despine(plt.gca())
plt.text(1, -14, 'R$^2$={0:.3f}'.format(r))
plt.title('Probability Plot')
plt.legend(loc='best', numpoints=4, handlelength=4);

#### with differencing

In [ ]:
carsales.diff(1).plot(label='1 period', title='Car sales')
plt.legend(loc='best')
despine(plt.gca())

In [ ]:
is_stationary(carsales.diff(1).dropna());

In [ ]:
carsales.diff(1).plot(label='1 period', title='Car sales',
                      dashes=(15,5))
carsales.diff(1).diff(12).plot(label='1 and 12 period(s)',
                               color='Coral')
plt.legend(loc='best')
despine(plt.gca())
plt.xlabel('Date')

In [ ]:
is_stationary(carsales.diff(1).diff(12).dropna());

## Time series models

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
is_stationary((carsales-carsales_seasonal).diff(1).dropna());

In [ ]:
ts = carsales-carsales_seasonal
tsdiff = ts.diff(1)

In [ ]:
plt.plot(tsdiff)

### Autoregressive model - AR

In [ ]:
model = ARIMA(ts, order=(1, 1, 0))  
arres = model.fit()

In [ ]:
arres.plot_predict(start='1961-12-01', end='1970-01-01', alpha=0.10)
plt.legend(loc='upper left')
print(arres.aic, arres.bic)

### Moving average model - MA

In [ ]:
model = ARIMA(ts, order=(0, 1, 1))  
mares = model.fit() 

In [ ]:
mares.plot_predict(start='1961-12-01', end='1970-01-01', alpha=0.10)
plt.legend(loc='upper left');
print(mares.aic, mares.bic)

### selecting p and q

In [ ]:
tsa.stattools.arma_order_select_ic(tsdiff.dropna(), max_ar=2, max_ma=2, ic='aic')

In [ ]:
acf = stt.acf(tsdiff.dropna(), nlags=10)
pacf = stt.pacf(tsdiff.dropna(), nlags=10)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,2))
ax1.axhline(y=0,color='gray')
ax1.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
ax1.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
ax1.axvline(x=1,ls=':',color='gray')
ax1.plot(acf)
ax1.set_title('ACF')

ax2.axhline(y=0,color='gray')
ax2.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
ax2.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
ax2.axvline(x=1,ls=':',color='gray')
ax2.plot(pacf)
ax2.set_title('PACF')

despine([ax1,ax2])

### Autoregressive Integrated Moving Average - ARIMA

In [ ]:
model = ARIMA(ts, order=(1, 0, 1))  
arimares = model.fit()

In [ ]:
arimares.plot_predict(start='1961-12-01', end='1970-01-01', alpha=0.10)
plt.legend(loc='upper left');
print(arimares.aic, arimares.bic)

In [74]:
arimares.forecast()

(array([ 18238.56843584]),
 array([ 1388.79665349]),
 array([[ 15516.57701316,  20960.55985853]]))

In [76]:
type(arimares.forecast())

tuple

In [83]:
arimares.predict(start=107)

/Users/santiago/anaconda/lib/python3.5/site-packages/statsmodels/base/data.py:503: FutureWarning: TimeSeries is deprecated. Please use Series
  return TimeSeries(result, index=self.predict_dates)


1968-12-01    18882.947811
Freq: MS, dtype: float64

In [81]:
len(ts)

108